In [1]:
import warnings
import os
import sys

warnings.filterwarnings('ignore')
current_dir = %pwd

parent_dir = os.path.abspath(os.path.join(current_dir, '../..'))
sys.path.append(parent_dir)

In [2]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from datasets import Dataset

checkpoint = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=5)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from src.main.models.roBERTa import RoBERTa
from src.main.pipeline.functions import clean_text, stop_words_removal
from src.main.utilities.utils import get_dataset
from sklearn.model_selection import train_test_split

roberta = RoBERTa()

roberta.pipeline = [clean_text, stop_words_removal]

inputs, targets = get_dataset(one_hot=True)
inputs = roberta.run_pipeline(inputs, save=True)

x_train, x_test, y_train, y_test = train_test_split(inputs, targets, test_size=0.2, random_state=42)

train_data = {'text': x_train, 'label': y_train.values.astype(float)}
val_data = {'text': x_test, 'label': y_test.values.astype(float)}

In [7]:
train_dataset = Dataset.from_dict(train_data)
val_dataset = Dataset.from_dict(val_data)

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map: 100%|██████████| 21648/21648 [00:05<00:00, 3895.87 examples/s]


In [8]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments("test-trainer")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)

trainer.train()

RuntimeError: result type Float can't be cast to the desired output type Long